In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [ ]:
blog_docs

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [ ]:
splits

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
huggingFaceApiToken = os.environ["HUGGINGFACEHUB_API_TOKEN"]
embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingFaceApiToken, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [ ]:
# Index
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

retriever = vectorstore.as_retriever(search_kwargs={"k": 1})